# OLD
# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff. TESTING NEW TIMESTEPS

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-11-22 07:25:55.272053: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 07:25:55.272199: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 07:25:55.276386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 07:25:55.562788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 07:25:57.642412: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_final_train_new_timesteps/")
ds_val = tf.data.Dataset.load("../data/ds_final_val_new_timesteps/")
ds_test = tf.data.Dataset.load("../data/ds_final_test_new_timesteps/")

2023-11-22 07:26:01.250238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 07:26:01.578435: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 07:26:01.578508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 07:26:01.582902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 07:26:01.582988: I tensorflow/compile

In [3]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [4]:
def transform_element(features, context, labels):
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-2], features[:,-2], features[:,-1]), labels[0]

In [5]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [6]:
for x, y in ds_train.take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(y.shape)

(64, 6)
(64,)
(64,)
(2,)


In [7]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.emebd_cont = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.conc = tf.keras.layers.Concatenate()
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, inputs):
        cont, plate, dd = inputs
        x1 = self.embed_dd(dd)
        x2 = self.embed_plate(plate)
        x3 = self.emebd_cont(cont)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [9]:
model = MyModel(64)

In [10]:
model.build(input_shape=[(None, 64, 6), (None, 64), (None, 64)])
model.count_params()

82498

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [12]:
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10


2023-11-22 07:26:16.387031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-11-22 07:26:17.631640: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7efd542118c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 07:26:17.631759: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2023-11-22 07:26:17.646390: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 07:26:17.815592: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proces

3227/3227 [==============================] - 343s 103ms/step - loss: 0.2076 - accuracy: 0.9398 - f1_score: 0.5125 - val_loss: 0.2225 - val_accuracy: 0.9377 - val_f1_score: 0.4839
Epoch 2/10
3227/3227 [==============================] - 452s 140ms/step - loss: 0.1979 - accuracy: 0.9412 - f1_score: 0.5504 - val_loss: 0.2157 - val_accuracy: 0.9415 - val_f1_score: 0.5644
Epoch 3/10
3227/3227 [==============================] - 387s 120ms/step - loss: 0.1696 - accuracy: 0.9473 - f1_score: 0.6471 - val_loss: 0.2137 - val_accuracy: 0.9410 - val_f1_score: 0.5570
Epoch 4/10
3227/3227 [==============================] - 311s 96ms/step - loss: 0.1590 - accuracy: 0.9481 - f1_score: 0.6676 - val_loss: 0.2105 - val_accuracy: 0.9411 - val_f1_score: 0.5500
Epoch 5/10
3227/3227 [==============================] - 306s 95ms/step - loss: 0.1556 - accuracy: 0.9488 - f1_score: 0.6741 - val_loss: 0.2092 - val_accuracy: 0.9414 - val_f1_score: 0.5603
Epoch 6/10
3227/3227 [==============================] - 304s 94

In [13]:
model.evaluate(ds_test)

 96/100 [===========================>..] - ETA: 0s - loss: 0.0889 - accuracy: 0.9797 - f1_score: 0.6826

100/100 [==============================] - 8s 82ms/step - loss: 0.0872 - accuracy: 0.9803 - f1_score: 0.6834


[0.0872003436088562,
 0.9802618026733398,
 array([0.98997205, 0.37682965], dtype=float32)]

In [14]:
model.save("../models/model_v2_new_ts.keras")

In [15]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v2_new_ts.csv", index=False)